In [ ]:
def gf16v_mul_256(a: int, b: int) -> int:
    """
    Multiplies a (256-bit) 'a' by a 4-bit 'b' under GF(16).
    
    :param a: 256-bit integer (Python int).
    :param b: 4-bit integer (0 <= b <= 0xF).
    :return:  256-bit integer result of the GF(16) multiply.
    """
    # For 256-bit, extend the "0x8 nibble" pattern across 64 hex digits.
    mask_msb = 0x8888888888888888888888888888888888888888888888888888888888888888

    # Ensure 'a' is in 256-bit range; ensure 'b' is in 4-bit range.
    a &= (1 << 256) - 1
    b &= 0xF

    result = 0

    # We have 4 bits in 'b', so iterate for i in {0,1,2,3}.
    for i in range(4):
        # If the i-th bit of b is set, XOR 'a' into result.
        if (b >> i) & 1:
            result ^= a

        # The standard GF(16) step:
        #  1) Extract top bits in each nibble (where bit 3 is set).
        a_msb = a & mask_msb
        #  2) Clear them from 'a'.
        a ^= a_msb
        #  3) Shift left by 1 (multiply by x).
        a <<= 1
        #  4) For all nibble MSBs that overflowed, shift them down by 3 bits
        #     to align with the nibble's LSB, then multiply by 3 and XOR.
        a ^= (a_msb >> 3) * 3

        # If you want to strictly stay within 256 bits, mask again:
        a &= (1 << 256) - 1

    # Mask result as well, if you want a strict 256-bit result.
    return result & ((1 << 256) - 1)

def main():
    # Example usage:
    # 256-bit 'a', with a sample big hex value:
    a_256 = 0x9ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF012345678
    # 4-bit 'b' in GF(16):
    b_4   = 0xF

    product = gf16v_mul_256(a_256, b_4)
    
    print(f"a   = 0x{a_256:064x}")
    print(f"b   = 0x{b_4:x}")
    print(f"r   = 0x{product:064x}")

if __name__ == "__main__":
    main()


a   = 0x0000000000000000000000000000000000000000000000009abcdef012345678
b   = 0x2
r   = 0x000000000000000000000000000000000000000000000000175b9fd02468ace3
